In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
                transforms.RandomVerticalFlip(),  
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05)
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 128
p1 = .25


class ResBlock(nn.Module):
    def __init__(self, in_size, kernel_size, stride, padding, padding_mode, encoder=False):
        super().__init__()
        if (encoder):
            self.conv1 = nn.Conv2d(in_size, in_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
        else:
            self.conv1 = nn.ConvTranspose2d(in_size, in_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(in_size)

    def convblock(self, x):
        x = F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        return x
   
    def forward(self, x): 
        return x + self.convblock(x)
    
    
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.resblock1 = ResBlock(in_size=1, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode, encoder=True)
        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.resblock2 = ResBlock(in_size=channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode, encoder=True)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.resblock3 = ResBlock(in_size=channels_base * 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.resblock4 = ResBlock(in_size=channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.resblock5 = ResBlock(in_size=channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm5 = nn.BatchNorm2d(2)
        
    def forward(self, input):
        # encoder
        x = self.resblock1(input)
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = y = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = self.resblock2(x)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = self.resblock3(x)
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = self.resblock4(x + y)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = self.resblock5(x + input)
        x = F.leaky_relu(self.batchnorm5(self.convtrans3(x)), negative_slope=0.1)


        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 1, 32, 32]              10
       BatchNorm2d-2            [-1, 1, 32, 32]               2
            Conv2d-3            [-1, 1, 32, 32]              10
       BatchNorm2d-4            [-1, 1, 32, 32]               2
          ResBlock-5            [-1, 1, 32, 32]               0
            Conv2d-6          [-1, 128, 32, 32]           1,280
       BatchNorm2d-7          [-1, 128, 32, 32]             256
            Conv2d-8          [-1, 128, 16, 16]         147,584
       BatchNorm2d-9          [-1, 128, 16, 16]             256
           Conv2d-10          [-1, 128, 16, 16]         147,584
      BatchNorm2d-11          [-1, 128, 16, 16]             256
         ResBlock-12          [-1, 128, 16, 16]               0
           Conv2d-13          [-1, 256, 16, 16]         295,168
      BatchNorm2d-14          [-1, 256,

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00253793 (0.01758111), PSNR 25.95519829 (20.36909739), SSIM 0.76284993 (0.64755832)
Finished training epoch 0


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 42 pixels
  return func(*args, **kwargs)


Validate: MSE 0.00324828 (0.00317392), PSNR 24.88346863 (25.08860989), SSIM 0.67967796 (0.74559589)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00203928 (0.00269833), PSNR 26.90524101 (25.71120008), SSIM 0.77154648 (0.76708509)
Finished training epoch 1


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 19 pixels
  return func(*args, **kwargs)


Validate: MSE 0.00310822 (0.00322265), PSNR 25.07487679 (25.03803221), SSIM 0.69674689 (0.75456002)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00235181 (0.00263524), PSNR 26.28598022 (25.81697653), SSIM 0.76176685 (0.76572658)
Finished training epoch 2


...

Validate: MSE 0.00303980 (0.00291308), PSNR 25.17154694 (25.45984603), SSIM 0.70565313 (0.76978798)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00296494 (0.00254556), PSNR 25.27984619 (25.96458436), SSIM 0.76593208 (0.76701181)
Finished training epoch 3
Validate: MSE 0.00301964 (0.00291265), PSNR 25.20044136 (25.46783589), SSIM 0.70848131 (0.77380934)
Finished validation.
Starting training epoch 4
Epoch: 4, MSE 0.00283403 (0.00249633), PSNR 25.47594833 (26.04835615), SSIM 0.73987210 (0.76952175)
Finished training epoch 4
Validate: MSE 0.00300523 (0.00291794), PSNR 25.22122192 (25.44838533), SSIM 0.70688242 (0.77038092)
Finished validation.
Starting training epoch 5
Epoch: 5, MSE 0.00282521 (0.00248104), PSNR 25.48949432 (26.07585504), SSIM 0.75200760 (0.77091244)
Finished training epoch 5
Validate: MSE 0.00295192 (0.00316241), PSNR 25.29895401 (25.14538279), SSIM 0.71082395 (0.77383652)
Finished validation.
Starting training epoch 6
Epoch: 6, MSE 0.00285302 (0.002452

Validate: MSE 0.00309473 (0.00267904), PSNR 25.09377098 (25.80159906), SSIM 0.71132010 (0.77856332)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00302668 (0.00223636), PSNR 25.19033051 (26.52741189), SSIM 0.78444707 (0.78346356)
Finished training epoch 33
Validate: MSE 0.00299915 (0.00264116), PSNR 25.23001480 (25.87651500), SSIM 0.71345323 (0.77996681)
Finished validation.
Starting training epoch 34
Epoch: 34, MSE 0.00221395 (0.00223608), PSNR 26.54832840 (26.53152850), SSIM 0.78188026 (0.78356649)
Finished training epoch 34
Validate: MSE 0.00341902 (0.00270999), PSNR 24.66098595 (25.74881900), SSIM 0.70980144 (0.77824110)
Finished validation.
Starting training epoch 35
Epoch: 35, MSE 0.00252996 (0.00223491), PSNR 25.96886444 (26.53067885), SSIM 0.75861949 (0.78367933)
Finished training epoch 35
Validate: MSE 0.00307583 (0.00268856), PSNR 25.12037468 (25.77910520), SSIM 0.70971382 (0.77802476)
Finished validation.
Starting training epoch 36
Epoch: 36, MSE 0.0022861

Validate: MSE 0.00319322 (0.00264855), PSNR 24.95771408 (25.84354440), SSIM 0.70411921 (0.77438652)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00232112 (0.00211710), PSNR 26.34302902 (26.76674645), SSIM 0.76486093 (0.78707444)
Finished training epoch 63
Validate: MSE 0.00308836 (0.00263070), PSNR 25.10272217 (25.87574211), SSIM 0.71175981 (0.77824291)
Finished validation.
Starting training epoch 64
Epoch: 64, MSE 0.00172685 (0.00212104), PSNR 27.62745285 (26.75690347), SSIM 0.79663920 (0.78725957)
Finished training epoch 64
Validate: MSE 0.00345453 (0.00279277), PSNR 24.61610985 (25.60932054), SSIM 0.69952309 (0.76994176)
Finished validation.
Starting training epoch 65
Epoch: 65, MSE 0.00162932 (0.00212073), PSNR 27.87994576 (26.75908617), SSIM 0.80039722 (0.78728134)
Finished training epoch 65
Validate: MSE 0.00314398 (0.00265923), PSNR 25.02519798 (25.82097572), SSIM 0.70788765 (0.77636862)
Finished validation.
Starting training epoch 66
Epoch: 66, MSE 0.0020871

Validate: MSE 0.00368684 (0.00297751), PSNR 24.33345985 (25.34245749), SSIM 0.69405329 (0.76724049)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00214655 (0.00204842), PSNR 26.68259048 (26.90812636), SSIM 0.80192244 (0.78879203)
Finished training epoch 93
Validate: MSE 0.00359899 (0.00274109), PSNR 24.43819618 (25.68852079), SSIM 0.70177370 (0.77412837)
Finished validation.
Starting training epoch 94
Epoch: 94, MSE 0.00205170 (0.00205316), PSNR 26.87887001 (26.89730348), SSIM 0.80528086 (0.78899871)
Finished training epoch 94
Validate: MSE 0.00353987 (0.00275530), PSNR 24.51012611 (25.66726662), SSIM 0.70187581 (0.77286634)
Finished validation.
Starting training epoch 95
Epoch: 95, MSE 0.00191285 (0.00205705), PSNR 27.18319130 (26.89363669), SSIM 0.79310971 (0.78867371)
Finished training epoch 95
Validate: MSE 0.00348995 (0.00271509), PSNR 24.57180786 (25.72998711), SSIM 0.69744921 (0.77462675)
Finished validation.
Starting training epoch 96
Epoch: 96, MSE 0.0020517

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00349236 (0.00271639), PSNR 24.56880569 (25.73413426), SSIM 0.69729930 (0.77469588)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()